## Env setup

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
# For emptying trash after each run
#from google.colab import auth
#auth.authenticate_user()
#from googleapiclient.discovery import build
#drive_service = build('drive', 'v3')
#drive_service.files().emptyTrash().execute()
!pwd
!nvidia-smi

/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/subtask2b
Tue Jan 30 12:30:52 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     On  | 00000000:2B:00.0 Off |                  N/A |
|  0%   38C    P8              26W / 350W |      3MiB / 12288MiB |      0%      Default |
|                                         |                      |                  

## Imports

In [3]:
folder_name = "/home/nlp-lab-ws23/nlp_praktikum/persuasion_technique_detection/" #"/content/drive/MyDrive/persuasion_technique_detection/" 

In [4]:
#!pip install transformers datasets wandb evaluate accelerate -qU sklearn_hierarchical_classification sentencepiece

In [5]:
import gc
import re
import json
import numpy as np
import pandas as pd
import random
import torch
import subprocess
import json
import warnings
import shutil
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AutoTokenizer, ViTFeatureExtractor
from sklearn.metrics import f1_score, accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer,LabelEncoder
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset,DatasetDict,concatenate_datasets
import datasets
import os
from torch.nn.functional import sigmoid
from datasets import concatenate_datasets
from transformers import Trainer
from PIL import Image
from transformers import AutoModel, AutoModelForSequenceClassification,AutoTokenizer,AutoFeatureExtractor,ViTImageProcessor,ViTConfig, BertConfig, VisionTextDualEncoderConfig, VisionTextDualEncoderModel,CLIPImageProcessor

In [6]:
import torch
AVAIL_GPUS = 0
if torch.cuda.is_available():
    device = torch.device("cuda")
    AVAIL_GPUS = torch.cuda.device_count()
    print(f'There are {AVAIL_GPUS} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3080 Ti


## Login WandB

In [7]:
import wandb
import os

#wandb.login(relogin=True)
wandb.login()

# setup wandb environment variables
os.environ['WANDB_PROJECT'] = "subtask2b"
os.environ['WANDB_ENTITY'] = "tumnlp"
os.environ["WANDB_LOG_MODEL"]= "end"

wandb: Currently logged in as: mahmudfami (tumnlp). Use `wandb login --relogin` to force relogin


In [8]:
text_checkpoint = "microsoft/deberta-v3-large"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(text_checkpoint)

/home/nlp-lab-ws23/nlp_praktikum/nlp_praktikum_env/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [10]:
summary_dir_path = folder_name + "subtask2b/new_summaries/summary_" + text_checkpoint.replace("/","_")+"/"

## Preprocess text input

In [11]:
val_path="data/subtask2b/val.json"
train_path="data/subtask2b/train.json"
test_path="data/subtask2b/dev_unlabeled.json"

with open(folder_name+val_path) as f:
  d = json.load(f)
  val=pd.DataFrame.from_dict(d)
  labels=val["label"]
  num_label=[int(el=="propagandistic") for el in labels]
  val["num_label"]=num_label
  val=val.drop(columns=['label'])
  val_set=val.rename(columns={"num_label": "label"})

with open(folder_name+train_path) as f:
  d = json.load(f)
  train=pd.DataFrame.from_dict(d)
  labels=train["label"]
  num_label=[int(el=="propagandistic") for el in labels]
  train["num_label"]=num_label
  train=train.drop(columns=['label'])
  train_set=train.rename(columns={"num_label": "label"})
  #mask = train_set['image'] == "prop_meme_24871.png"
  #train_set = train_set[~mask]

with open(folder_name+test_path) as f:
  d = json.load(f)
  dev_unlabeled_set=pd.DataFrame.from_dict(d)

label2num={"non_propagandistic":0,"propagandistic":1}
num2label={0:"non_propagandistic",1:"propagandistic"}

print(len(train_set),len(val_set),len(dev_unlabeled_set))


1200 150 300


In [12]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, dataset_type):
        super().__init__()
        self.ids=list(dataset["id"])
        self.texts = list(dataset["text"])
        self.image_paths = list(dataset["image"])
        if dataset_type=="train" or dataset_type=="val":
          self.labels = dataset["label"].astype(int).tolist()
        self.dataset_type=dataset_type

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):


        if self.dataset_type=="train":
          image_path=folder_name+ "data/subtask2b/subtask2b_images/train/"+ self.image_paths[idx]
        elif self.dataset_type=="val":
          image_path=folder_name+ "data/subtask2b/subtask2b_images/val/"+ self.image_paths[idx]
        else:
          image_path=folder_name+ "data/subtask2b/subtask2b_images/dev/"+ self.image_paths[idx]


        if self.dataset_type=="train" or self.dataset_type=="val":
          label = torch.tensor(self.labels[idx], dtype=torch.float32)
          return self.ids[idx],self.texts[idx],label
        else:
          return self.ids[idx],self.texts[idx]

In [13]:
# Create a custom dataset
train_dataset = CustomDataset(train_set,"train" )
val_dataset = CustomDataset(val_set,"val")
test_dataset = CustomDataset(dev_unlabeled_set,"test")

batch_size=2
num_workers=2
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers,pin_memory=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,num_workers=num_workers,pin_memory=True,drop_last=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,num_workers=num_workers,pin_memory=True)

In [14]:
print("train size:",len(train_set),"val size:",len(val_set),"test size:",len(dev_unlabeled_set))

train size: 1200 val size: 150 test size: 300


## Training

In [15]:
transfer_learning = False
model_nodes = {
    "persuasion_or_not":None}
parent_nodes = {
    "persuasion_or_not":"persuasion_or_not"
}

In [16]:
def train():
  try:
    # Initialize a new wandb run
    wandb.init()

    # sweep agent inputs config with hyperparameters
    config = wandb.config

    learning_rate = config.learning_rate
    run_name = config.run_name+f"_{str(learning_rate)}learningRate"
    wandb.run.name = run_name

    num_epochs = 10
    text_model = AutoModelForSequenceClassification.from_pretrained(text_checkpoint, num_labels=2 ,id2label=num2label,label2id=label2num,ignore_mismatched_sizes=True)

    text_model.cuda()
    loss_func = nn.BCEWithLogitsLoss()
    optimizer = optim.AdamW(text_model.parameters(), lr=learning_rate)
    best_val_f1_macro=0.
    for epoch in range(num_epochs):
        text_model.train()
        total_loss = 0.0
        all_preds = []
        all_labels = []
        print("Started training epoch:",epoch+1)
        for id, text_input, label in train_dataloader:
            optimizer.zero_grad()
            text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)

            text_input=text_input.to("cuda")
            label=label.to("cuda")

            logits =text_model(**text_input).logits
            pred_logits,_=torch.max(logits,dim=1)
            loss = loss_func(pred_logits, label)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

            # Collect predictions and labels for metrics calculation
            preds_classes = torch.argmax(logits,dim=1).cpu().numpy()

            all_preds.extend(preds_classes)
            all_labels.extend(label.cpu().numpy())

        average_loss = total_loss / len(train_dataloader)

        # Calculate and log F1 score and accuracy using sklearn metrics
        f1_macro = f1_score(all_labels, all_preds,average="macro")
        f1_micro = f1_score(all_labels, all_preds,average="micro")
        accuracy = accuracy_score(all_labels, all_preds)
        # Log metrics to W&B
        wandb.log({"epoch": epoch + 1, "train_loss": average_loss, "f1_macro": f1_macro, "f1_micro":f1_micro, "train_accuracy": accuracy})

        # Validation
        text_model.eval()
        val_loss = 0.0
        all_val_preds = []
        all_val_labels = []
        with torch.no_grad():
            for id, text_input, label in val_dataloader:
                text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)

                text_input=text_input.to("cuda")
                label=label.to("cuda")

                logits =text_model(**text_input).logits
                pred_logits,_=torch.max(logits,dim=1)

                val_loss = loss_func(pred_logits, label).item()
                # Collect predictions and labels for metrics calculation
                val_preds_classes = torch.argmax(logits,dim=1).cpu().numpy()
                all_val_preds.extend(val_preds_classes)
                all_val_labels.extend(label.cpu().numpy())
        average_val_loss = val_loss / len(val_dataloader)

        # Calculate and log F1 score and accuracy for validation
        val_f1_macro = f1_score(all_val_labels, all_val_preds,average="macro")
        val_f1_micro = f1_score(all_val_labels, all_val_preds,average="micro")
        val_accuracy = accuracy_score(all_val_labels, all_val_preds)
        # Log metrics to W&B
        wandb.log({"epoch": epoch + 1, "val_loss": average_val_loss, "val_f1_macro": val_f1_macro,"val_f1_micro":val_f1_micro, "val_accuracy": val_accuracy})

        # Save the model if the current validation f1_macro is better than the previous best
        if val_f1_macro > best_val_f1_macro:
          best_val_f1_macro = val_f1_macro
          best_model_state_dict = text_model.state_dict()
          wandb.log({"eval/f1_macro":val_f1_macro})

        print(f"Epoch {epoch + 1}/{num_epochs}, Training f1-macro: {f1_macro}, Validation f1-macro: {val_f1_macro}")
        text_model.train()
    # Finish W&B run

    artifact = wandb.Artifact(f"best_model_{run_name}".replace("/","_"), type="model")
    artifact.add_file(folder_name+f"best_model_{run_name}.pth".replace("/","_"), torch.save(best_model_state_dict, folder_name+f"best_model_{run_name}.pth".replace("/","_")))
    wandb.run.log_artifact(artifact)
    os.remove(folder_name+f"best_model_{run_name}.pth".replace("/","_"))
    #drive_service.files().emptyTrash().execute()
    wandb.finish()
  except Exception as e:
    print(f"Error in training: {str(e)}")

In [17]:
# Set hyperparams in sweep configurations
run_name=f'{text_checkpoint}-TEXTONLY-subtask2b-memes'.replace("/","_")
sweep_name=f'sweep_{run_name}'
sweep_config = {
    'method': 'grid',  # can be grid, random, or bayes
    'name' : sweep_name,
    'metric': {
      'name': 'eval/f1_macro',
      'goal': 'maximize'
    },
    'parameters': {
        'learning_rate' : {
            'values': [1e-6,3e-6,5e-6,5e-5,5e-4]
        },
        'run_name': {
            'value' : run_name
        }
    }
}

# Start sweeps with specific configuration
sweep_id = wandb.sweep(sweep_config, project="subtask2b")
wandb.agent(sweep_id, train)
# Get best model of sweep
api = wandb.Api()
sweep = api.sweep(f"subtask2b/{sweep_id}")
best_run = sweep.best_run()

artifacts = best_run.logged_artifacts()

model_artifact = None
for artifact in artifacts:
    if 'model' in artifact.type:  # Adjust the condition based on your setup
        model_artifact = artifact
        break

if model_artifact != None:
  model_artifact_name = model_artifact.name
  print(f"Best Model: {model_artifact_name}")
else:
  warnings.warn(f"No models was found")

# save best model of this node
model_nodes["persuasion_or_not"] = model_artifact_name


Create sweep with ID: ioun9tls
Sweep URL: https://wandb.ai/tumnlp/subtask2b/sweeps/ioun9tls


wandb: Agent Starting Run: 3zoyllcj with config:
wandb: 	learning_rate: 0.0005
wandb: 	run_name: microsoft_deberta-v3-large-TEXTONLY-subtask2b-memes-BigLR


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Started training epoch: 1
Epoch 1/10, Training f1-macro: 0.4287272542394816, Validation f1-macro: 0.4
Started training epoch: 2
Epoch 2/10, Training f1-macro: 0.4, Validation f1-macro: 0.4
Started training epoch: 3
Epoch 3/10, Training f1-macro: 0.4, Validation f1-macro: 0.4
Started training epoch: 4
Epoch 4/10, Training f1-macro: 0.42054728372977596, Validation f1-macro: 0.4
Started training epoch: 5
Epoch 5/10, Training f1-macro: 0.4, Validation f1-macro: 0.4
Started training epoch: 6
Epoch 6/10, Training f1-macro: 0.48852947324414714, Validation f1-macro: 0.4
Started training epoch: 7
Epoch 7/10, Training f1-macro: 0.4511574381445581, Validation f1-macro: 0.25
Started training epoch: 8
Epoch 8/10, Training f1-macro: 0.4338459238775014, Validation f1-macro: 0.4
Started training epoch: 9
Epoch 9/10, Training f1-macro: 0.47405329593267886, Validation f1-macro: 0.25
Started training epoch: 10
Epoch 10/10, Training f1-macro: 0.4513362894459877, Validation f1-macro: 0.4


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
eval/f1_macro,▁
f1_macro,▃▁▁▃▁█▅▄▇▅
f1_micro,███▇█▅▁█▄▇
train_accuracy,███▇█▅▁█▄▇
train_loss,█▃▃▃▂▃▅▁▁▂
val_accuracy,██████▁█▁█
val_f1_macro,██████▁█▁█
val_f1_micro,██████▁█▁█
val_loss,▅▁▂▃▄█▇▆▂▃
epoch,10


...

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Started training epoch: 1
Epoch 1/10, Training f1-macro: 0.42355706958638484, Validation f1-macro: 0.4
Started training epoch: 2
Epoch 2/10, Training f1-macro: 0.4184936215720724, Validation f1-macro: 0.4
Started training epoch: 3
Epoch 3/10, Training f1-macro: 0.42834139415030376, Validation f1-macro: 0.4
Started training epoch: 4
Epoch 4/10, Training f1-macro: 0.4019522807757369, Validation f1-macro: 0.4
Started training epoch: 5
Epoch 5/10, Training f1-macro: 0.4, Validation f1-macro: 0.4
Started training epoch: 6
Epoch 6/10, Training f1-macro: 0.4, Validation f1-macro: 0.4
Started training epoch: 7
Epoch 7/10, Training f1-macro: 0.4, Validation f1-macro: 0.4
Started training epoch: 8
Epoch 8/10, Training f1-macro: 0.39969984992496244, Validation f1-macro: 0.4
Started training epoch: 9
Epoch 9/10, Training f1-macro: 0.5021071172192346, Validation f1-macro: 0.4
Started training epoch: 10
Epoch 10/10, Training f1-macro: 0.4, Validation f1-macro: 0.4


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇██
eval/f1_macro,▁
f1_macro,▃▂▃▁▁▁▁▁█▁
f1_micro,▇█▇▇████▁█
train_accuracy,▇█▇▇████▁█
train_loss,█▂▄▂▁▁▁▂▄▁
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_f1_macro,▁▁▁▁▁▁▁▁▁▁
val_f1_micro,▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▅▆▅▅▅▁▅▆
epoch,10


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
wandb: Sorting runs by -summary_metrics.eval/f1_macro


Best Model: best_model_microsoft_deberta-v3-large-TEXTONLY-subtask2b-memes-BigLR_0.0005learningRate:v0


In [ ]:
wandb.finish()

## Eval on val set and save run

In [18]:
def write_json(path,data,file_name="summary.json"):
  if not isinstance(data, dict):
    data = data.to_dict("records")
  if not os.path.exists(path):
    os.makedirs(path)
  with open(path+file_name, "w") as output_file:
      json.dump(data, output_file, indent=2,ensure_ascii=False)

In [19]:

summary = {
    "text_checkpoint" : text_checkpoint,
    "model_nodes" : model_nodes,
    "train_path" : train_path,
    "val_path":val_path,
    "test_path":test_path
}
print(summary)

{'text_checkpoint': 'microsoft/deberta-v3-large', 'model_nodes': {'persuasion_or_not': 'best_model_microsoft_deberta-v3-large-TEXTONLY-subtask2b-memes-BigLR_0.0005learningRate:v0'}, 'train_path': 'data/subtask2b/train.json', 'val_path': 'data/subtask2b/val.json', 'test_path': 'data/subtask2b/dev_unlabeled.json'}


In [20]:
write_json(summary_dir_path,summary)

### Eval on val set

In [21]:
api = wandb.Api()
artifact=api.artifact(model_nodes["persuasion_or_not"])
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, model_nodes["persuasion_or_not"].split(":")[0]+".pth" )
model_state_dict = torch.load(model_state_dict_path)

text_model = AutoModelForSequenceClassification.from_pretrained(text_checkpoint, num_labels=2 ,id2label=num2label,label2id=label2num,ignore_mismatched_sizes=True)
text_model.load_state_dict(model_state_dict)
text_model.cuda()

text_model.eval()

wandb: Downloading large artifact best_model_microsoft_deberta-v3-large-TEXTONLY-subtask2b-memes-BigLR_0.0005learningRate:v0, 1659.85MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.9
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, element

In [22]:
loss_func = nn.BCEWithLogitsLoss()
val_loss = 0.0
all_val_preds = []
all_val_labels = []

with torch.no_grad():
    for id, text_input, label in val_dataloader:

        text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)
        text_input=text_input.to("cuda")
        label=label.to("cuda")

        logits =text_model(**text_input).logits
        pred_logits,_=torch.max(logits,dim=1)

        val_loss = loss_func(pred_logits, label).item()
        # Collect predictions and labels for metrics calculation
        val_preds_classes = torch.argmax(logits,dim=1).cpu().numpy()
        all_val_preds.extend(val_preds_classes)
        all_val_labels.extend(label.cpu().numpy())

average_val_loss = val_loss / len(val_dataloader)

# Calculate and log F1 score and accuracy for validation
val_f1_macro = f1_score(all_val_labels, all_val_preds,average="macro")
val_f1_micro = f1_score(all_val_labels, all_val_preds,average="micro")
val_accuracy = accuracy_score(all_val_labels, all_val_preds)
val_results={"f1_macro":val_f1_macro,"f1_micro":val_f1_micro,"val_accuracy":val_accuracy}
print(val_results)
summary["val_results"]=val_results
print(summary)

val_pred_labels=[num2label[el.item()] for el in all_val_preds]
val_dataset_=val_set.drop(columns=["label","text","image"])
val_dataset_["label"]=val_pred_labels
write_json(summary_dir_path,val_dataset_,"val_preds.json")

{'f1_macro': 0.4, 'f1_micro': 0.6666666666666666, 'val_accuracy': 0.6666666666666666}
{'text_checkpoint': 'microsoft/deberta-v3-large', 'model_nodes': {'persuasion_or_not': 'best_model_microsoft_deberta-v3-large-TEXTONLY-subtask2b-memes-BigLR_0.0005learningRate:v0'}, 'train_path': 'data/subtask2b/train.json', 'val_path': 'data/subtask2b/val.json', 'test_path': 'data/subtask2b/dev_unlabeled.json', 'val_results': {'f1_macro': 0.4, 'f1_micro': 0.6666666666666666, 'val_accuracy': 0.6666666666666666}}


In [23]:
write_json(summary_dir_path, summary)

In [24]:
def delete_dir(path):
    if os.path.exists(path):
        try:
            shutil.rmtree(path)
            print(f"The directory {path} and all its contents have been deleted successfully")
        except OSError as e:
            print(f"Error: {e.strerror}")
    else:
        print(f"The directory {path} does not exist")

In [25]:
def delete_output_dirs(parent_directory):
    for entry in os.listdir(parent_directory):
        entry_path = os.path.join(parent_directory, entry)

        if os.path.isdir(entry_path) and entry.startswith('output_'):
            delete_dir(entry_path)

In [ ]:
path = folder_name + "subtask2b"
delete_output_dirs(path)
delete_dir(path + "tumnlp")
delete_dir(path + "wandb")
delete_dir(path + "artifacts")
delete_dir(path + "tmp_trainer")

## Predict on test set

In [26]:
api = wandb.Api()
artifact=api.artifact(model_nodes["persuasion_or_not"])
model_dir=artifact.download()
model_state_dict_path = os.path.join(model_dir, model_nodes["persuasion_or_not"].split(":")[0]+".pth" )
model_state_dict = torch.load(model_state_dict_path)

text_model = AutoModelForSequenceClassification.from_pretrained(text_checkpoint, num_labels=2 ,id2label=num2label,label2id=label2num,ignore_mismatched_sizes=True)
text_model.load_state_dict(model_state_dict)
text_model.cuda()

text_model.eval()

wandb: Downloading large artifact best_model_microsoft_deberta-v3-large-TEXTONLY-subtask2b-memes-BigLR_0.0005learningRate:v0, 1659.85MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:2.1
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.weight', 'classifier.bias', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, element

In [27]:
all_preds = []

with torch.no_grad():
  for id, text_input in test_dataloader:
    text_input = tokenizer(text_input, return_tensors="pt",truncation=True, padding=True,max_length=512)
    text_input=text_input.to("cuda")
    label=label.to("cuda")

    logits =text_model(**text_input).logits
    test_preds_classes = torch.argmax(logits,dim=1).cpu().numpy()

    all_preds.extend(test_preds_classes)

test_pred_labels=[num2label[el.item()] for el in all_preds]
dev_unlabeled_set["label"]=test_pred_labels
dev_unlabeled_set=dev_unlabeled_set.drop(columns=["text","image"])
write_json(summary_dir_path,dev_unlabeled_set,"dev_preds.json")

